In [ ]:
#%matplotlib notebook

import time
import numpy as np

In [ ]:
from collections import OrderedDict
from reachy import parts

def patch_right_arm_config(arm_cls):
    arm_cls.dxl_motors = OrderedDict([
         ('right_arm.shoulder_pitch', {
             'id': 10, 'offset': 90.0, 'orientation': 'indirect',
             'angle-limits': [-180, 60],
             'link-translation': [0, -0.19, 0], 'link-rotation': [0, 1, 0],
         }),
         ('right_arm.shoulder_roll', {
             'id': 11, 'offset': 90.0, 'orientation': 'indirect',
             'angle-limits': [-100, 90],
             'link-translation': [0, 0, 0], 'link-rotation': [1, 0, 0],
         }),
         ('right_arm.arm_yaw', {
             'id': 12, 'offset': 0.0, 'orientation': 'indirect',
             'angle-limits': [-90, 90],
             'link-translation': [0, 0, 0], 'link-rotation': [0, 0, 1],
         }),
         ('right_arm.elbow_pitch', {
             'id': 13, 'offset': 0.0, 'orientation': 'indirect',
             'angle-limits': [0, 125],
           'link-translation': [0, 0, -0.28], 'link-rotation': [0, 1, 0],
         }),
    ])

    return arm_cls

def patch_left_arm_config(arm_cls):
    arm_cls.dxl_motors = OrderedDict([
         ('left_arm.shoulder_pitch', {
             'id': 20, 'offset': -90.0, 'orientation': 'indirect',
             'angle-limits': [-180, 60],
             'link-translation': [0, -0.19, 0], 'link-rotation': [0, 1, 0],
         }),
         ('left_arm.shoulder_roll', {
             'id': 21, 'offset': -90.0, 'orientation': 'indirect',
             'angle-limits': [-100, 90],
             'link-translation': [0, 0, 0], 'link-rotation': [1, 0, 0],
         }),
         ('left_arm.arm_yaw', {
             'id': 22, 'offset': 0.0, 'orientation': 'indirect',
             'angle-limits': [-90, 90],
             'link-translation': [0, 0, 0], 'link-rotation': [0, 0, 1],
         }),
         ('left_arm.elbow_pitch', {
             'id': 23, 'offset': 0.0, 'orientation': 'indirect',
             'angle-limits': [0, 125],
           'left_arm.link-translation': [0, 0, -0.28], 'link-rotation': [0, 1, 0],
         }),
    ])

    return arm_cls

def patch_force_gripper(forceGripper):
    def __init__(self, root, io):
        """Create a new Force Gripper Hand."""
        parts.hand.Hand.__init__(self, root=root, io=io)

        dxl_motors = OrderedDict({
            name: dict(conf)
            for name, conf in self.dxl_motors.items()
        })

        self.attach_dxl_motors(dxl_motors)

        """
        self._load_sensor = self.io.find_module('force_gripper')
        self._load_sensor.offset = 4
        self._load_sensor.scale = 10000
        """

    forceGripper.__init__ = __init__

    return forceGripper

In [ ]:
# Connect to arms

In [ ]:
from reachy import Reachy, parts

In [ ]:
parts.LeftArm = patch_left_arm_config(parts.LeftArm)

In [ ]:
parts.RightArm = patch_right_arm_config(parts.RightArm)

In [ ]:
parts.arm.LeftForceGripper = patch_force_gripper(parts.arm.LeftForceGripper)

In [ ]:
parts.arm.RightForceGripper = patch_force_gripper(parts.arm.RightForceGripper)

In [ ]:
reachy = Reachy(
    left_arm=parts.LeftArm(io='/dev/ttyUSB*', hand='force_gripper'),
    right_arm=parts.RightArm(io='/dev/ttyUSB*', hand='force_gripper'),
)

## Turn stiff

In [ ]:
for m in reachy.left_arm.motors:
    m.compliant = False
for m in reachy.right_arm.motors:
    m.compliant = False

## Turn compliant

In [ ]:
for m in reachy.left_arm.motors:
    m.compliant = True
for m in reachy.right_arm.motors:
    m.compliant = True

# Record Gesture

In [ ]:
from reachy.trajectory import TrajectoryRecorder, TrajectoryPlayer

In [ ]:
recorder = TrajectoryRecorder(reachy.left_arm.motors, reachy.right_arm.motors)

We will record a move for 10sec. Make sure you hold the robot away from all objects before starting the record.


In [ ]:
recorder.start()
time.sleep(10)
recorder.stop()

In [ ]:
## Turn Stiff
for m in reachy.left_arm.motors:
    m.compliant = False
for m in reachy.right_arm.motors:
    m.compliant = False

In [ ]:
player = TrajectoryPlayer(reachy, recorder.trajectories)
player.play(wait=True, fade_in_duration=3)